In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install fastai --upgrade

Requirement already up-to-date: fastai in /usr/local/lib/python3.6/dist-packages (2.1.8)


In [3]:
import sys
sys.path.append("/content/drive/MyDrive/ELEC 4240/Final Version")

In [4]:
import os
import glob
import time
import numpy as np
import torch
from torch import nn, optim
import matplotlib.pyplot as plt

from model import MainModel
from model_cycleGAN import MainModel_cycleGAN
from functions import train_model, test_model, train_model_cycleGAN, test_model_cycleGAN
from data_loader import make_dataloaders
from generator_res_unet import build_res_unet, pretrain_generator
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
path = "/content/drive/MyDrive/ELEC 4240/Final Version/flower_500"
test_path = "/content/drive/MyDrive/ELEC 4240/Final Version/flower_test_sketch"


paths = glob.glob(path + "/*.jpg") # Grabbing all the image file names
test_paths = glob.glob(test_path + "/*.jpg") # Grabbing all the image file names
np.random.seed(123)
paths_subset = np.random.choice(paths, 500, replace=False) # choosing 1000 images randomly
rand_idxs = np.random.permutation(500)
train_idxs = rand_idxs[:450] # choosing the first 8000 as training set
val_idxs = rand_idxs[450:] # choosing last 2000 as validation set
train_paths = paths_subset[train_idxs]
val_paths = paths_subset[val_idxs]
train_dl = make_dataloaders(paths=train_paths, split='train')
val_dl = make_dataloaders(paths=val_paths, split='val')
np.random.seed(123)
test_paths_subset = np.random.choice(test_paths, 3, replace=False)
rand_idxs = np.random.permutation(3)
test_idxs = rand_idxs[:3]
TEST_paths = test_paths_subset[test_idxs]
test_dl = make_dataloaders(paths=TEST_paths, split='test')
data = next(iter(train_dl))
Ls, abs_ = data['L'], data['ab']
data = next(iter(val_dl))
Ls, abs_ = data['L'], data['ab']
data = next(iter(test_dl))
Ls, abs_ = data['L'], data['ab']

In [6]:
# cGAN unet
intend_model = "unet"

loss_D_fake = []
loss_D_real = []
loss_D = []
loss_G_GAN = []
loss_G_L1 = []
loss_G = []

if (intend_model == "unet"):
    model_x = MainModel()
    loss_D_fake, loss_D_real, loss_D, loss_G_GAN, loss_G_L1, loss_G = train_model(model_x,train_dl, val_dl, 101)
    fig = plt.figure()
    ax = fig.add_subplot(231)
    ax.set_xlabel("Iterations")
    ax.set_ylabel("unet loss_D_fake")
    ax.plot(loss_D_fake)
    bx = fig.add_subplot(232)
    bx.set_xlabel("Iterations")
    bx.set_ylabel("unet loss_D_real")
    bx.plot(loss_D_real)
    cx = fig.add_subplot(233)
    cx.set_xlabel("Iterations")
    cx.set_ylabel("unet loss_D")
    cx.plot(loss_D)
    dx = fig.add_subplot(234)
    dx.set_xlabel("Iterations")
    dx.set_ylabel("unet loss_G_GAN")
    dx.plot(loss_G_GAN)
    ex = fig.add_subplot(235)
    ex.set_xlabel("Iterations")
    ex.set_ylabel("unet loss_G_L1")
    ex.plot(loss_G_L1)
    fx = fig.add_subplot(236)
    fx.set_xlabel("Iterations")
    fx.set_ylabel("unet loss_G")
    fx.plot(loss_G)
    fig.tight_layout()
    test_model(model_x,test_dl)

Output hidden; open in https://colab.research.google.com to view.

In [7]:
# pretrain res18_unet generator
net_G = build_res_unet(n_input=1, n_output=2, size=256)
opt = optim.Adam(net_G.parameters(), lr=1e-4)
criterion = nn.L1Loss()

pretrain_generator(net_G, train_dl, opt, criterion, 20)
torch.save(net_G.state_dict(), "res18-unet.pt")


Epoch 1/20
L1 Loss: 0.16220



Epoch 2/20
L1 Loss: 0.13963



Epoch 3/20
L1 Loss: 0.13706



Epoch 4/20
L1 Loss: 0.13639



Epoch 5/20
L1 Loss: 0.13607


...


Epoch 16/20
L1 Loss: 0.13138



Epoch 17/20
L1 Loss: 0.13119



Epoch 18/20
L1 Loss: 0.13051



Epoch 19/20
L1 Loss: 0.12898



Epoch 20/20
L1 Loss: 0.12843


In [8]:
# cGAN res18_unet
intend_model = "res_unet"

loss_D_fake = []
loss_D_real = []
loss_D = []
loss_G_GAN = []
loss_G_L1 = []
loss_G = []

if (intend_model == "res_unet"):
    net_G = build_res_unet(n_input=1, n_output=2, size=256)
    net_G.load_state_dict(torch.load("res18-unet.pt", map_location=device))
    model = MainModel(net_G=net_G)
    loss_D_fake, loss_D_real, loss_D, loss_G_GAN, loss_G_L1, loss_G = train_model(model, train_dl, val_dl, 101)
    fig = plt.figure()
    ax = fig.add_subplot(231)
    ax.set_xlabel("Iterations")
    ax.set_ylabel("res_unet loss_D_fake")
    ax.plot(loss_D_fake)
    bx = fig.add_subplot(232)
    bx.set_xlabel("Iterations")
    bx.set_ylabel("res_unet loss_D_real")
    bx.plot(loss_D_real)
    cx = fig.add_subplot(233)
    cx.set_xlabel("Iterations")
    cx.set_ylabel("res_unet loss_D")
    cx.plot(loss_D)
    dx = fig.add_subplot(234)
    dx.set_xlabel("Iterations")
    dx.set_ylabel("res_unet loss_G_GAN")
    dx.plot(loss_G_GAN)
    ex = fig.add_subplot(235)
    ex.set_xlabel("Iterations")
    ex.set_ylabel("res_unet loss_G_L1")
    ex.plot(loss_G_L1)
    fx = fig.add_subplot(236)
    fx.set_xlabel("Iterations")
    fx.set_ylabel("res_unet loss_G")
    fx.plot(loss_G)
    fig.tight_layout()
    test_model(model,test_dl)

Output hidden; open in https://colab.research.google.com to view.

In [9]:
# cycleGAN res18_unet
loss_D_b = []
loss_D_a = []
loss_G = []

net_G_a2b = build_res_unet(n_input=1, n_output=2, size=256)
net_G_a2b.load_state_dict(torch.load("res18-unet.pt", map_location=device))
net_G_b2a = build_res_unet(n_input=1, n_output=2, size=256)
net_G_b2a.load_state_dict(torch.load("res18-unet.pt", map_location=device))

model_yy = MainModel_cycleGAN(net_G_a2b=net_G_a2b, net_G_b2a=net_G_b2a)
loss_D_b, loss_D_a, loss_G = train_model_cycleGAN(model_yy,train_dl, val_dl, 101)
fig = plt.figure()
ax = fig.add_subplot(131)
ax.set_xlabel("Iterations")
ax.set_ylabel("res_unet loss_D_b")
ax.plot(loss_D_b)
bx = fig.add_subplot(132)
bx.set_xlabel("Iterations")
bx.set_ylabel("res_unet loss_D_a")
bx.plot(loss_D_a)
cx = fig.add_subplot(133)
cx.set_xlabel("Iterations")
cx.set_ylabel("res_unet loss_G")
cx.plot(loss_G)
fig.tight_layout()
test_model_cycleGAN(model_yy,test_dl)

Output hidden; open in https://colab.research.google.com to view.

In [10]:
# end!!